In [2]:
# import dill
import scanpy as sc
import os
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np

# import pyranges
# Set stderr to null to avoid strange messages from ray
import sys
_stderr = sys.stderr
null = open(os.devnull,'wb')
from scipy.sparse import issparse


import itertools
import anndata
import itertools
import anndata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import warnings
warnings.simplefilter(action='ignore')
import numpy as np
from sklearn.preprocessing import normalize

from itertools import product
import umap
from collections import Counter
from sklearn.decomposition import PCA
from gridot.utils import *
from gridot.train import *

In [3]:
atac_anndata = sc.read_h5ad('./data/pseudo_atac_pbmc_binarized_20_20.h5ad')
atac_anndata

AnnData object with n_obs × n_vars = 2664 × 197784
    obs: 'celltype', 'n_genes', 'orig.ident'
    var: 'Chromosome', 'Start', 'End', 'n_cells', 'features', 'chr_no', 'start', 'end'

In [4]:
atac_anndata.var['chr_no'] = atac_anndata.var['Chromosome']
atac_anndata.var['start'] = atac_anndata.var['Start']
atac_anndata.var['end'] = atac_anndata.var['End']

In [5]:
rna_anndata = sc.read_h5ad('./data/adata.h5ad')

# Check if raw data exists
if rna_anndata.raw is not None:
    # Access the raw data
    raw_data = rna_anndata.raw.X
    raw_obs = rna_anndata.obs.copy()  # Cell metadata (unchanged)
    raw_var = rna_anndata.raw.var.copy()  # Gene metadata (original raw genes)

    # Create a new AnnData object using the raw data
    rna_anndata = sc.AnnData(X=raw_data, obs=raw_obs, var=raw_var)

    # Print the new raw AnnData object to verify
    print(rna_anndata)
else:
    print("The AnnData object does not contain raw data.")

AnnData object with n_obs × n_vars = 2664 × 21255
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'ingest_celltype_label', 'leiden_res_0.8', 'celltype'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'


In [9]:
import os

schema_reference = 'rna'
marker_gene = 'CD38'
distance_threshold = 1e5
device = 'cuda:0'
num_components = 50
gene_file = "./data/gencode.v38.annotation.gff3"
base_folder_path = './testing_results_pseudo'
# seeds = [1, 2, 3]

# Preprocess once
atac_adata, rna_adata = preprocessing_adata(atac_anndata, rna_anndata, gene_file=gene_file)
print("Processing multimodal")
preprocess_multimodal(rna_adata, atac_adata)

# Run for each seed
# for seed in seeds:
# seed_folder_path = os.path.join(base_folder_path, f"seed_{seed}")
os.makedirs(base_folder_path, exist_ok=True)
print(f"Running CRE-gene relationship analysis:")
run_gridot_cell_population(
    rna_adata,
    atac_adata,
    schema_reference=schema_reference,
    marker_gene=marker_gene,
    distance_threshold=distance_threshold,
    device=device,
    folder_path=base_folder_path,
    seed=1
)


Pre-processing scATAC-seq...
AnnData object with n_obs × n_vars = 2664 × 197784
    obs: 'celltype', 'n_genes', 'orig.ident'
    var: 'Chromosome', 'Start', 'End', 'n_cells', 'features', 'chr_no', 'start', 'end'
Pre-processing scRNA-seq...
AnnData object with n_obs × n_vars = 2664 × 21167
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'ingest_celltype_label', 'leiden_res_0.8', 'celltype'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'features'
Adding start site to meta file...
Processing multimodal
Running CRE-gene relationship analysis:
Identifying candidate pairs within 100000.0 bp
463449
Running quadratic program... done.
Cell with highest CD38 expression: ('CTAGCTTGTTGAGGTC-1', 1124)
Expression level:   (0, 0)	5.438936233520508
Epoch 0 (35.78 seconds): train loss 1885.62
Epoch Inference (57.54 seconds): test loss 1786.72
Total Time: 96.312330